# Quickstart Dashboard 入门指南

本笔记帮助你快速体验 `notebooks/quickstart_dashboard.py` 中提供的轻量级统计仪表盘框架。
通过少量代码即可把 Git2Base 的分析结果渲染成可交互的项目/运行选择页面，并在“全部”及各技术栈标签页查看单次运行卡片，在“趋势分析”标签中对比历史指标。

## 1. 环境准备

确保你已经在 `artifacts/` 目录中生成了至少一次分析运行的数据。
"
    "目录结构应类似：
"
    "```
artifacts/
  your-project/
    runs/
      20240101-120000/
        data/
          commit_files.csv
          diff_results.csv
          ...
```
"
    "这些 CSV 文件会被仪表盘加载并转换成 Pandas DataFrame。

## 2. 引入仪表盘框架

In [ ]:
from quickstart_dashboard import (
    DiffFileChangeCountStatistic,
    QuickstartDashboard,
    RecordCountStatistic,
    load_statistic_from_notebook,
)

## 3. 创建仪表盘实例
默认情况下，框架会从 `../artifacts` 读取数据。
如果你的数据位于其他位置，可以使用 `RunDataLoader` 指定新的路径。

> 如果你的技术栈配置文件不在默认的 `config/config.yaml` 路径，可以通过 `config_path` 参数传入自定义 YAML。

In [ ]:
# 将 base_dir 替换成你实际存放分析结果的目录（相对于当前 notebook）。
dashboard = QuickstartDashboard(base_dir="../artifacts")
# 你也可以显式传入 loader：
# loader = RunDataLoader(base_dir="/path/to/artifacts")
# dashboard = QuickstartDashboard(loader=loader)


## 4. 注册基础统计
`RecordCountStatistic` 会根据运行类型自动选择合适的 CSV 表并统计记录数，方便快速校验仪表盘。
`DiffFileChangeCountStatistic` 针对 diff 运行统计新增、变更、删除文件数量，snapshot 运行会显示为 N/A。
如果你还想查看字符区间分布，可在后续步骤通过加载器引入 `FileCharCountDistributionStatistic` 自定义统计。

In [ ]:
dashboard.register_statistic(
    RecordCountStatistic()
)
dashboard.register_statistic(
    DiffFileChangeCountStatistic()
)

## 5. 自定义你的统计卡片

在 `notebooks/custom_statistics/` 目录下创建专用的开发 notebook，使用其中的可视化单元调试统计逻辑。完成后，通过下面的加载器将 notebook 中的 `BaseStatistic` 子类引入到当前 quickstart 环境。

In [ ]:
# 使用加载器将开发 notebook 中的类引入当前会话。
# 仅预览使用的单元格可在 metadata.tags 中添加 'skip-loader'，
# 以避免通过加载器导入时执行这些辅助代码。
# 如果 notebook 中定义了多个统计类，可以传入 class_name 参数进行选择。
FileCharCountDistributionStatistic = load_statistic_from_notebook(
    "custom_statistics/file_char_count_distribution_statistic.ipynb"
)
UniqueAuthorsStatistic = load_statistic_from_notebook(
    "custom_statistics/unique_authors_statistic.ipynb"
)

In [ ]:
# 注册自定义统计（可按需添加多个）
dashboard.register_statistic(FileCharCountDistributionStatistic())
dashboard.register_statistic(UniqueAuthorsStatistic())

## 6. 展示仪表盘
运行下方单元格，即可在 Jupyter 中看到交互式仪表盘。
当你新增或删除运行结果后，可调用 `dashboard.refresh()` 重新加载数据。

> 技术栈切换：仪表盘会读取 `config/config.yaml` 中的 `stacks` 配置，并在每个仓库下按技术栈拆分标签页。 可以在这些标签页中查看所选运行的单次结果；而“趋势分析”标签提供技术栈、统计种类、执行类型与运行筛选，用于对比历史趋势。

In [ ]:
dashboard.show()
# 如需刷新数据：
# dashboard.refresh()
